In [1]:
from pypdf import PdfReader
import os, re
import pandas as pd
from pathlib import Path

import sys
sys.path.insert(1, '../src/')
import gemini_wrapper as gemini

In [2]:
def iterate_folders(root, on_each):
    for p in Path(root).rglob('*'):
        if p.is_dir():
            iterate_folders(str(p), on_each)
        else:            
            on_each(str(p))

def process_file_metrics(i):
    min_thres = 200
    separator = '\n$$$\n'
    txt = open(f'../BData/parsed_data/{i}.txt').read()
    pages = txt.split(separator)
    max_page = max([len(page) for page in pages])
    min_page = min([len(page) for page in pages])
    exclud_pages = sum([len(page) < min_thres for page in pages])
    #expags = [i for page, i in zip(pages, range(len(pages))) if len(page) < min_thres]
    #expags_list.append(expags)
    #print(f'doc {i}, total pages {len(pages)}, max chars in page {max_page}, min chars page: {min_page}, excluded pages {exclud_pages}')
    return len(pages), max_page, min_page, exclud_pages

def process_file(i):
    min_thres = 200
    separator = '\n$$$\n'
    txt = open(f'../BData/parsed_data/{i}.txt').read()
    pages = txt.split(separator)

    idxs = []
    valid_pages = []
    for i in range(len(pages)):
        if len(pages[i]) > min_thres:
            idxs.append(i)
            valid_pages.append(pages[i])

    return valid_pages, idxs

In [4]:
root = '../BData'
files = []
iterate_folders(root, lambda p: files.append(p))
len(files)

1918

In [6]:
#pdfs = [f for f in files if re.match()]
f = files[0]
ptrn = '.*\\.pdf$'
pdf_paths = [f for f in files if re.match(ptrn, f)]
print(len(pdf_paths))
pdf_path_table = [[i, pdf_paths[i]] for i in range(len(pdf_paths))]
df = pd.DataFrame(pdf_path_table, columns=['id', 'path'])
df.to_csv('../metadata/pdf_path_table.csv', index=None)

609


In [ ]:
separator = '\n$$$\n'
for i in range(len(pdf_paths)):
    pdf_path = pdf_paths[i]
    reader = PdfReader(pdf_path)
    number_of_pages = len(reader.pages)
    txt = separator.join([page.extract_text() for page in reader.pages])
    open(f'../BData/parsed_data/{i}.txt', 'w+').write(txt)

print('finished')

finished


In [ ]:
mtx = []
for i in range(len(pdf_paths)):
    pages, max_page, min_page, exclud_pages = process_file_metrics(i)
    mtx.append([i, pages, max_page, min_page, exclud_pages])

In [19]:
df = pd.DataFrame(mtx, columns=['id', 'pages', 'min_page', 'max_page', 'exclud_pages'])

In [20]:
df.sort_values(['pages'])

,id,pages,min_page,max_page,exclud_pages
15,15,1,2801,2801,0
16,16,1,1883,1883,0
20,20,1,4940,4940,0
21,21,1,5136,5136,0
22,22,1,4092,4092,0
...,...,...,...,...,...
252,252,136,7909,0,1
306,306,136,7909,0,1
605,605,136,7909,0,1
551,551,136,7909,0,1


In [34]:
df.head(8)

,id,pages,min_page,max_page,exclud_pages
0,0,37,3019,332,0
1,1,1,3625,3625,0
2,2,10,5962,81,2
3,3,71,3154,346,0
4,4,1,1101,1101,0
5,5,592,4845,0,6
6,6,22,3018,37,5
7,7,8,2062,366,0


In [24]:
# Full Data Processing Gemini

In [28]:
gemini.configure()

In [30]:
txts = [
    'Hi, my name is Vagner',
    'What is going on?'
]
embs = gemini.create_embedding(txts)

In [56]:
emb_dict = []
for i in range(len(pdf_paths)):
    pages, idxs = process_file(i)
    print(f'creating emb {i}')        
    embs = gemini.create_embedding(pages)
    emb_dict.append([pages, idxs, embs])

print('finished')

creating emb 0
creating emb 1
creating emb 2
creating emb 3
creating emb 4
creating emb 5
creating emb 6
creating emb 7
creating emb 8
creating emb 9
creating emb 10
creating emb 11
creating emb 12
creating emb 13
creating emb 14
creating emb 15
creating emb 16
creating emb 17
creating emb 18
creating emb 19
creating emb 20
creating emb 21
creating emb 22
creating emb 23
creating emb 24
creating emb 25
creating emb 26
creating emb 27
creating emb 28
creating emb 29
creating emb 30
creating emb 31
creating emb 32
creating emb 33
creating emb 34
creating emb 35
creating emb 36
creating emb 37
creating emb 38
creating emb 39
creating emb 40
creating emb 41
creating emb 42
creating emb 43
creating emb 44
creating emb 45
creating emb 46
creating emb 47
creating emb 48
creating emb 49
creating emb 50
creating emb 51
creating emb 52
creating emb 53
creating emb 54
creating emb 55
creating emb 56
creating emb 57
creating emb 58
creating emb 59
creating emb 60
creating emb 61
creating emb 62
cr

In [57]:
corpus = []
for doc, (pages, idxs, embs) in zip(range(len(emb_dict)), emb_dict):
    for txt, page, emb in zip(pages, idxs, embs):
        corpus.append(
            {
                'txt':txt,
                'emb': emb,
                'doc':doc,
                'page':page
            }
        )

In [58]:
import json
open('../tmp/full_corpus.json', 'w+').write(json.dumps(corpus))

83875104